In [7]:
!pip install corus -q
!pip install pymorphy2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.7/83.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Задание 1

__Выберите 5 неоднозначных лексем и соберите для них словарные значения (толкования). Можно использовать МАС или любой другой толковый словарь. Обязательно укажите, каким словарем вы пользовались. - 1 балл за ручной сбор данных, 1 балл за краулинг__

Я пользовалась словарём МАС.

In [8]:
dict_of_words = {
    'ключ': [
        'Металлическое приспособление для запирания и отпирания замка',
        'Орудие, приспособление для укрепления или отвинчивания чего-л., для приведения в действие механизмов различного рода',
        'Средство, возможность для разгадки, понимания чего-л., для овладения чем-л.',
        'Пункт, место, важные в военном отношении, овладение которыми открывает доступ куда-л., обеспечивает победу',
        'Знак в начале нотной строки, условно указывающий на ноту, по высоте которой устанавливается высотное положение других нот (муз.)',
        'Бьющий из земли источник, родник',
        'Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи',
        'Верхний или средний камень, которым замыкается свод или арка здания'

    ],
    'мешать': [
        'Быть, являться помехой, препятствием в чем-л.',
        'Перемешивать, размешивать при помощи ложки, мешалки и т. п.',
        'Соединять в одно что-л. разнородное',
        'Путать, принимать одно за другое.'
    ],
    'печь': [
        'Приготовлять пищу сухим нагреванием на жару',
        'Каменное или металлическое сооружение для отопления помещений, приготовления горячей пищи.',
        'Обдавать сильным жаром, зноем'
    ],
    'работа': [
        'деятельность, занятие, труд',
        'Та или иная деятельность по созданию, изготовлению, обработке чего-л',
        'Служба, занятие, труд на каком-л. предприятии, в каком-л. учреждении',
        'То, что подлежит обработке, находится в процессе изготовления',
        'Продукт труда, изделие, произведение',
        'Качество или какой-л. способ, манера исполнения, изготовления',
        'Мера действия силы на тело, перемещаемое этой силой по определенному пути'
    ],
    'среда': [
        'Вещество, заполняющее какое-л. пространство и обладающее определенными свойствами.',
        'Совокупность природных или социальных условий, в которых протекает жизнедеятельность какого-л. организма.',
        'Социально-бытовая обстановка, в которой протекает жизнь человека; окружение',
        'Третий (после воскресенья) день недели (середина недели)'
    ]
}

## Задание 2

__Выберите один или несколько корпусов коллекции Corus, извлеките оттуда все предложения, где содержатся ваши неоднозначные слова. - 1 балл__

In [9]:
from tqdm import tqdm
from corus import load_lenta, load_morphoru_rnc
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
from nltk.tokenize import RegexpTokenizer, sent_tokenize

import nltk
nltk.download('punkt')

tokenizer = RegexpTokenizer('\w+')
mystem = Mystem()

def tokenize(text):
    return tokenizer.tokenize(text)
def lemmatized_context(s):
    return [w.lower() for w in mystem.lemmatize(" ".join(tokenize(s)))]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


Я выбрала словари новостей Ленты и морфологически размеченный корпус НКРЯ.

In [ ]:
!wget https://github.com/dialogue-evaluation/morphoRuEval-2017/raw/master/RNC_texts.rar
!unrar x RNC_texts.rar
!rm RNC_texts.rar

In [ ]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
!tar -xzvf lenta-ru-news.tar.gz

In [12]:
path = '/content/lenta-ru-news.csv.gz'
records_lenta = load_lenta(path)

In [13]:
path = '/content/RNCgoldInUD_Morpho.conll'
record_rnc = load_morphoru_rnc(path)

In [ ]:
dict_of_sent = {word: {'text': [], 'lemmatized_text': []} for word in dict_of_words.keys()}
lengths = {word: 0 for word in dict_of_words.keys()}

for record in tqdm(record_rnc):
    for token in record.tokens:
        if token.lemma in dict_of_words.keys():
            try:
                sent = ' '.join([token.text for token in record.tokens])
                lemmatized_text = ' '.join([token.lemma for token in record.tokens])
                dict_of_sent[token.lemma]['text'].append(sent)
                dict_of_sent[token.lemma]['lemmatized_text'].append(lemmatized_text)
                lengths[token.lemma] += 1
            except Exception as e:
                print(e)
                print(' '.join([token.text for token in record.tokens]))

for word in dict_of_sent.keys():
    print(word, len(dict_of_sent[word]['text']))

for record in tqdm(records_lenta):
    sentences = sent_tokenize(record.text)
    for sent in sentences:
        lemmatized_text = lemmatized_context(sent)
        for word in dict_of_words.keys():
            if word in lemmatized_text:
                dict_of_sent[word]['text'].append(sent)
                dict_of_sent[word]['lemmatized_text'].append(lemmatized_text)
                lengths[word] += 1

    # делаю ограничение, чтобы для каждого слова было минимум 80 примеров,
    # так как код работает долго
    if all([x >= 80 for x in lengths.values()]):
        break

In [15]:
for word in dict_of_sent.keys():
    print(word, len(dict_of_sent[word]['text']))

ключ 169
мешать 221
печь 80
работа 4336
среда 1181


## Задание 3

__Разбейте полученные контексты по значениям двумя любыми способами (вы можете использовать способы с семинара, заменить там эмбеддинги, реализовать какой-то свой способ). Напоминалка: в случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo и другими контекстуальными эмбеддингами — нужно кластеризовать. - 2 балла__

In [16]:
!pip install tensorflow Cython matplotlib -q
!pip install git+https://github.com/lopuhin/python-adagram.git -q
!pip install simple-elmo -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.7 MB/s eta 0:00:00


In [17]:
import nltk
nltk.download('stopwords')

import adagram
from simple_elmo import ElmoModel

import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

morph = MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian'))

def normalize(text):
    words = [morph.parse(word)[0].normal_form for word in tokenize(text) if word]
    return words

def tokenize(text):
    return token.tokenize(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### AdaGram

In [18]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100.nonorm.slim.joblib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  45.0M      0  0:00:30  0:00:30 --:--:-- 44.6M


In [19]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [20]:
def disambiguate(model, word, context):
    word, _ = lemmatized_context(word)
    probs = model.disambiguate(word, lemmatized_context(context))
    return 1 + probs.argmax()

С помощью модели для каждого слова предсказываю номер значения. То же самое повторяю и для словарных определений.

In [21]:
for word in dict_of_sent.keys():
    dict_of_sent[word]['sense'] = []
for word in dict_of_sent.keys():
    for sent in tqdm(dict_of_sent[word]['text']):
        dict_of_sent[word]['sense'].append(disambiguate(vm, word, sent))

  0%|          | 0/169 [00:00<?, ?it/s]

  0%|          | 0/221 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/4336 [00:00<?, ?it/s]

  0%|          | 0/1181 [00:00<?, ?it/s]

#### Задание 4 для AdaGram
__Возьмите словарные толкования для каждого значения и припишите им значение автоматически теми же способами, что и в п.2. - 1 балл__

In [22]:
sense2ind = {key: {} for key in dict_of_words.keys()}
for word in dict_of_words.keys():
    senses = []
    for sense in tqdm(dict_of_words[word]):
        ind = disambiguate(vm, word, sense)
        if ind not in sense2ind[word]:
            sense2ind[word][ind] = []
        sense2ind[word][ind].append(sense)

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

### Elmo

In [23]:
model = ElmoModel()
path = '/content/drive/MyDrive/NLP Course/196'
model.load(path)

'The model is now loaded.'

In [24]:
def get_elmo_vectors(word, contexts, model):
    tokens = [normalize(c) for c in contexts]
    all_vectors = model.get_elmo_vectors(tokens)
    word_vecs = []
    error_idx = []
    for i in range(len(contexts)):
        try:
            word_vecs.append(all_vectors[i][tokens[i].index(word)])
        except ValueError:  # если нормализация накосячила и лемму не найти
            print(word, 'error id', i)
            error_idx.append(i)
    return word_vecs, error_idx

Получаю вектора слов для каждого из контекстов

In [25]:
for word in tqdm(dict_of_sent.keys()):
    sents = dict_of_sent[word]['text']
    dict_of_sent[word]['vectors'], dict_of_sent[word]['errors'] = get_elmo_vectors(word, sents, model)


  0%|          | 0/5 [00:00<?, ?it/s]

печь error id 30
печь error id 38
печь error id 54
печь error id 56
печь error id 64
печь error id 65
печь error id 70
печь error id 72
печь error id 73
печь error id 74
печь error id 77
печь error id 78


In [26]:
# pymorphy в некоторых контекстах не смог получить "печь", удаляю их
dict_of_sent['печь']['text'] = pd.Series(dict_of_sent['печь']['text']).drop(dict_of_sent['печь']['errors'])

Дальше я кластеризую примеры и считаю центроиды кластеров. Я выбрала метод KMeans, так как он позволяет выбрать количество определений, что удобно, ведь по словарной статье мы предполагаем нужное количество.

In [27]:
centroids = {}
for word in dict_of_sent.keys():
    X = dict_of_sent[word]['vectors'].copy()
    db = KMeans(n_clusters=len(dict_of_words[word])).fit(X)
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

    X = np.array(X)
    word_centroids = {}
    for label in set(labels):
        word_centroids[label] = X[labels==label].mean(axis=0)
    centroids[word] = word_centroids

    dict_of_sent[word]['labels_elmo'] = labels

#### Задание 4 для ELMO
__Возьмите словарные толкования для каждого значения и припишите им значение автоматически теми же способами, что и в п.2. - 1 балл__

Дальше я считаю средний вектор для каждого из определений в словаре и выбираю ближайший центроид по косинусной близости. Номер этого центроида и есть необходимое значение.

In [28]:
sense2ind_elmo = {}
for word, senses in tqdm(dict_of_words.items()):
    tokens = [normalize(sense) for sense in senses]
    tokens = [word] + tokens
    all_vectors = model.get_elmo_vectors(tokens).mean(axis=1)
    if word not in sense2ind_elmo:
        sense2ind_elmo[word] = {}
    for i, sense in enumerate(senses):
        label_best, cos_best = -2, -2
        for label, vector in centroids[word].items():
            cos_value = cosine(all_vectors[i], vector)
            if cos_value > cos_best and label:
                cos_best = cos_value
                label_best = label
        if label_best not in sense2ind_elmo[word]:
            sense2ind_elmo[word][label_best] = []
        sense2ind_elmo[word][label_best].append(sense)

  0%|          | 0/5 [00:00<?, ?it/s]

In [29]:
sense2ind_elmo

{'ключ': {4: ['Металлическое приспособление для запирания и отпирания замка'],
  5: ['Орудие, приспособление для укрепления или отвинчивания чего-л., для приведения в действие механизмов различного рода',
   'Средство, возможность для разгадки, понимания чего-л., для овладения чем-л.',
   'Пункт, место, важные в военном отношении, овладение которыми открывает доступ куда-л., обеспечивает победу',
   'Знак в начале нотной строки, условно указывающий на ноту, по высоте которой устанавливается высотное положение других нот (муз.)',
   'Бьющий из земли источник, родник',
   'Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи',
   'Верхний или средний камень, которым замыкается свод или арка здания']},
 'мешать': {3: ['Быть, являться помехой, препятствием в чем-л.'],
  2: ['Перемешивать, размешивать при помощи ложки, мешалки и т. п.'],
  1: ['Соединять в одно что-л. разнородное',
   'Путать, принимать одно за другое.']},
 'печь': {1: ['При

In [30]:
sense2ind

{'ключ': {1: ['Металлическое приспособление для запирания и отпирания замка'],
  2: ['Орудие, приспособление для укрепления или отвинчивания чего-л., для приведения в действие механизмов различного рода',
   'Знак в начале нотной строки, условно указывающий на ноту, по высоте которой устанавливается высотное положение других нот (муз.)',
   'Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи'],
  3: ['Средство, возможность для разгадки, понимания чего-л., для овладения чем-л.',
   'Пункт, место, важные в военном отношении, овладение которыми открывает доступ куда-л., обеспечивает победу'],
  4: ['Бьющий из земли источник, родник',
   'Верхний или средний камень, которым замыкается свод или арка здания']},
 'мешать': {1: ['Быть, являться помехой, препятствием в чем-л.',
   'Соединять в одно что-л. разнородное'],
  2: ['Перемешивать, размешивать при помощи ложки, мешалки и т. п.'],
  3: ['Путать, принимать одно за другое.']},
 'печь': {

In [31]:
dict_of_sent['печь']['text'] = dict_of_sent['печь']['text'].tolist()

In [32]:
import pandas as pd

data = pd.DataFrame(columns=['word', 'text', 'sense_ada_id', 'sense_ada_text', 'sense_elmo_id', 'sense_elmo_text'])
for word, values in dict_of_sent.items():
    for i in range(len(values['text'])):
        sense_ada_id = values['sense'][i]
        sense_ada_text = sense2ind[word].get(values['sense'][i], None)
        sense_elmo_id = values['labels_elmo'][i]
        sense_elmo_text = sense2ind_elmo[word].get(values['labels_elmo'][i], None)
        data = data.append({
            'word': word,
            'text': dict_of_sent[word]['text'][i],
            'sense_ada_id': sense_ada_id,
            'sense_ada_text': sense_ada_text,
            'sense_elmo_id': sense_elmo_id,
            'sense_elmo_text': sense_elmo_text
        }, ignore_index=True)

In [33]:
for word in dict_of_words.keys():
    print(word)
    print(data[(data['word']==word) & (~data['sense_ada_text'].isna())].sense_ada_id.value_counts())

ключ
2    39
3    25
1    25
4    25
Name: sense_ada_id, dtype: int64
мешать
1    124
3     64
2     33
Name: sense_ada_id, dtype: int64
печь
4    30
3    10
2     7
Name: sense_ada_id, dtype: int64
работа
4    1686
1    1320
Name: sense_ada_id, dtype: int64
среда
2    820
3    122
4     91
Name: sense_ada_id, dtype: int64


In [34]:
for word in dict_of_words.keys():
    print(word)
    print(data[(data['word']==word) & (~data['sense_elmo_text'].isna())].sense_elmo_id.value_counts())

ключ
4    22
5     8
Name: sense_elmo_id, dtype: int64
мешать
3    68
1    61
2    38
Name: sense_elmo_id, dtype: int64
печь
1    10
Name: sense_elmo_id, dtype: int64
работа
5    594
2    298
Name: sense_elmo_id, dtype: int64
среда
2    321
Name: sense_elmo_id, dtype: int64


## Задание 5

__Выберите по 2-3 значения для слова и по 5 контекстов, которые автоматически были отнесены к этим значениям. Разметьте контексты: совпадает ли значение слова в контексте со словарным значением. - 1 балл__


Пример того, как я случайно выбирала примеры по значениям

In [43]:
bake_senses = data[(data['word']=='печь') & (data['sense_ada_id']==2)].sample(5)
bake_senses = bake_senses.append(data[(data['word']=='печь') & (data['sense_ada_id']==4)].sample(5))
bake_senses = bake_senses.append(data[(data['word']=='печь') & (data['sense_ada_id']==3)].sample(5))

In [44]:
bake_senses.index

Int64Index([450, 442, 449, 434, 455, 396, 399, 392, 410, 415, 431, 426, 454,
            447, 433],
           dtype='int64')

In [70]:
keys_senses_idx = [107, 81, 84, 56, 138, 164, 165, 14, 19, 162, 157, 82, 0, 68, 42]
mix_senses_idx = [329, 372, 306, 206, 282, 259, 251, 217, 288, 375, 188, 349, 209, 198, 261]
bake_senses_idx = [450, 442, 449, 434, 455, 396, 399, 392, 410, 415, 431, 426, 454, 447, 433]
work_senses_idx = [3395, 1119, 2221, 3885, 2466, 4448, 1423, 648, 4724, 1259]
env_senses_idx = [5948, 5091, 5847, 5296, 5059, 4830, 5817, 4898, 5940, 5598, 5601,4984, 4840, 4928, 5495]

all_idx = keys_senses_idx + mix_senses_idx + bake_senses_idx + work_senses_idx + env_senses_idx
data.loc[all_idx, ['word', 'text', 'sense_ada_text', 'sense_ada_id']].to_excel('ada_results_new.xlsx')

In [69]:
keys_senses_idx = [34, 38, 141, 120, 68, 86, 25, 40, 21, 11, 135, 49, 76, 166, 43]
mix_senses_idx = [197, 232, 185, 377, 178, 299, 322, 384, 330, 278, 331, 226, 332, 335, 295]
bake_senses_idx = [393, 413, 401, 439, 399]
work_senses_idx = [ 666, 3696,  721, 2683, 3933, 3051, 3153, 2376, 4135, 1481, 4704, 3740, 3161, 4586, 2718]
env_senses_idx = [5002, 5756, 5713, 5042, 5778, 5513, 5411, 5204, 5170, 5795, 2199, 2666, 1894, 2255, 2527]

all_idx = keys_senses_idx + mix_senses_idx + bake_senses_idx + work_senses_idx + env_senses_idx
data.loc[all_idx, ['word', 'text', 'sense_elmo_text', 'sense_elmo_id']].to_excel('elmo_results.xlsx')

## Задание 6

__Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает с автоматически определенным значением толкования. - 1 балл__

В столбце true_label проставлены метки из словаря в начале тетрдаки - номер значения слова, который мне показался подходящим.

In [43]:
pd.set_option('max_colwidth', None)

ada_results_path = '/content/drive/MyDrive/NLP Course/ada_results.xlsx'
elmo_results_path = '/content/drive/MyDrive/NLP Course/elmo_results.xlsx'

ada_results = pd.read_excel(ada_results_path).drop(columns=['Unnamed: 0'])
elmo_results = pd.read_excel(elmo_results_path).drop(columns=['Unnamed: 0'])

In [44]:
ada_results.head()

,word,text,sense_ada_text,sense_ada_id,true_label,is_true
0,ключ,"Порой непродуманные решения в данном ключе приобретали самые гипертрофированные формы, вплоть до вырубки виноградников, что на долгие годы практически уничтожило целые отрасли сельского хозяйства в ряде стран», — добавили в администрации президента республики.","['Бьющий из земли источник, родник', 'Верхний или средний камень, которым замыкается свод или арка здания']",4,3,no
1,ключ,"27 октября было открыто движение поездов по одному пути между станциями Туапсе и Горячий Ключ, отметили в РЖД.","['Бьющий из земли источник, родник', 'Верхний или средний камень, которым замыкается свод или арка здания']",4,6,yes
2,ключ,Губернатор Московской области Андрей Воробьев вручил представителям подмосковных муниципалитетов ключи от новой дорожно-уборочной техники.,"['Бьющий из земли источник, родник', 'Верхний или средний камень, которым замыкается свод или арка здания']",4,1,no
3,ключ,Он был известен по фильмам «Освобождение» и «Ключи от неба».,"['Бьющий из земли источник, родник', 'Верхний или средний камень, которым замыкается свод или арка здания']",4,1,no
4,ключ,"Ключ от мира находится в Кремле», — цитирует его «Интерфакс».","['Бьющий из земли источник, родник', 'Верхний или средний камень, которым замыкается свод или арка здания']",4,1,no


In [4]:
elmo_results.head()

,word,text,sense_elmo_text,sense_elmo_id,true_label,is_true
0,ключ,"Возвращаясь к основной нашей теме , ещё раз подчеркнём : информационные технологии многократно увеличили возможности доступа к текстам , но сами по себе они не дают ключа к той информации , которая содержится в этих текстах .","['Бьющий из земли источник, родник', 'Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи', 'Верхний или средний камень, которым замыкается свод или арка здания']",3,3,no
1,ключ,В то же время описанные выше факты и события дают ключ к объяснению таких удивительных явлений как психофизический парадокс и феномен ретроактивности .,"['Бьющий из земли источник, родник', 'Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи', 'Верхний или средний камень, которым замыкается свод или арка здания']",3,3,no
2,ключ,"«Я всегда знал, что ключ к успеху в любом деле — это вера в свои силы.","['Бьющий из земли источник, родник', 'Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи', 'Верхний или средний камень, которым замыкается свод или арка здания']",3,3,no
3,ключ,"По мнению британской стороны, установление того, с кем они встречались в Женеве, теперь является «абсолютным ключом к продолжающемуся расследованию».","['Бьющий из земли источник, родник', 'Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи', 'Верхний или средний камень, которым замыкается свод или арка здания']",3,3,no
4,ключ,"«Восстановление экономики, восстановление социальной сферы, перспективные проекты, которые можно и нужно восстановить, — это тоже ключ к возобновлению нормальной жизни в Ливии», — сказал Медведев на конференции по Ливии, которая проходит в Италии.","['Бьющий из земли источник, родник', 'Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи', 'Верхний или средний камень, которым замыкается свод или арка здания']",3,3,no


In [45]:
acc_ada = (ada_results.is_true == 'yes').sum() / ada_results.shape[0]
print('Accuracy of AdaGram:', acc_ada)

Accuracy of AdaGram: 0.5


In [46]:
for word in dict_of_words.keys():
    true_cnt = ((ada_results.word == word) & (ada_results.is_true == 'yes')).sum()
    all_cnt = (ada_results.word == word).sum()
    print(word, true_cnt/all_cnt)

ключ 0.6
мешать 0.3333333333333333
печь 0.26666666666666666
работа 0.5
среда 0.8


In [47]:
acc_elmo = (elmo_results.is_true == 'yes').sum() / elmo_results.shape[0]
print('Accuracy of Elmo:', acc_elmo)

Accuracy of Elmo: 0.4166666666666667


In [48]:
for word in dict_of_words.keys():
    true_cnt = ((elmo_results.word == word) & (elmo_results.is_true == 'yes')).sum()
    all_cnt = (elmo_results.word == word).sum()
    print(word, true_cnt/all_cnt)

ключ 0.2
мешать 0.3333333333333333
печь 1.0
работа 0.3333333333333333
среда 0.7


## Задание 7

__Сравните результаты двух методов WSD, объясните, почему результаты совпадают или расходятся (и покажите на конкретных примерах) - максимум 2 балла__

Обе модели сработали не очень хорошо... Но АдаГрам немного лучше. Основная проблема заключается в том, что понимание слов моделью сильно отличается от словарной статьи. Из этого следовало то, что модели присваивали одно значение разным определениям из статьи. А каким-то определениям не доставалось ни одной метки.

Когда я расставляла оценки по выборке из примеров, я ставила пометку "yes", если верное определение содержится в списке тех определений, которые модель объединила под одним лейблом. Например, ELMO объединила 5 значений слова "работа" под одной меткой - 2.

Трудно сравнивать результаты моделей с реальными ответами, так как нельзя с уверенностью сказать, что модель смогла выделить значение, если она отнесла его к некоторому списку значений. И между собой так же трудно, так как они так же по-разному считывают значения.

#### 1. Среда
Со средой у обеих моделей получилось в целом хорошо. Они обе отлично разделили понятия среды как дня недели и среды как пространства некоторого. Более глубокое разделение обеим моделям достаётся с трудом.

In [86]:
data[(data['word']=='среда') & (~data['sense_elmo_text'].isna()) & (~data['sense_ada_text'].isna())].sample(5)

,word,text,sense_ada_id,sense_ada_text,sense_elmo_id,sense_elmo_text
5650,среда,"Группа парламентариев во главе со спикером Совета Федерации России Валентиной Матвиенко направила в Госдуму законопроект, позволяющий полицейским отслеживать россиян по телефону без санкции суда, пишет «Коммерсантъ» в среду, 12 сентября.",2,[Третий (после воскресенья) день недели (середина недели)],3,"[Совокупность природных или социальных условий, в которых протекает жизнедеятельность какого-л. организма., Социально-бытовая обстановка, в которой протекает жизнь человека; окружение, Третий (после воскресенья) день недели (середина недели)]"
5688,среда,"Представители Климатического партнерства (инициатива ряда российских компаний, объединивших усилия бизнеса по сокращению воздействия на окружающую среду и предотвращению климатических изменений) отметили, что приоритетным направлением взаимодействия бизнеса и государства является безболезненное внедрение «зеленых технологий» и углеродного регулирования, поскольку государство в одиночку, без поддержки бизнеса, не сможет двигаться в направлении декарбонизации экономики, а это единственный способ сохранить свою конкурентоспособность на мировом рынке.",4,"[Социально-бытовая обстановка, в которой протекает жизнь человека; окружение]",1,"[Вещество, заполняющее какое-л. пространство и обладающее определенными свойствами.]"
5895,среда,"Следующая часть спецпроекта «Рабы божьи», посвященная экономическим тайнам РПЦ, выйдет в среду, 8 августа.",2,[Третий (после воскресенья) день недели (середина недели)],3,"[Совокупность природных или социальных условий, в которых протекает жизнедеятельность какого-л. организма., Социально-бытовая обстановка, в которой протекает жизнь человека; окружение, Третий (после воскресенья) день недели (середина недели)]"
5677,среда,"«Например, если брать отрасль, за которую я отвечаю, мы сейчас готовим аналогичный проект по кадровому резерву в сфере развития городской среды и ЖКХ.",4,"[Социально-бытовая обстановка, в которой протекает жизнь человека; окружение]",1,"[Вещество, заполняющее какое-л. пространство и обладающее определенными свойствами.]"
5638,среда,"«Нормативная среда на сегодня не учитывает ключевых свойств новой экономики — высокой ценности нематериальных активов, размывания границ и предельно низкие затраты на тиражирование бизнес-моделей, в том числе регуляторные вопросы, которые должны были быть решены еще вчера: определение юридической природы новых цифровых активов, идентификация субъектов цифровой экономики, юридическая сила электронных документов, формы государственного контроля бизнеса, регулирование оборота данных и доступа к ним таким образом, чтобы они были защищены и в то же время доступны для решения различных бизнес-задач», — сказал старший вице-президент, заместитель председателя совета директоров, ГМК «Норильский никель», вице-президент ЗАО «Холдинговая компания ИНТЕРРОС» Андрей Бугров.",4,"[Социально-бытовая обстановка, в которой протекает жизнь человека; окружение]",1,"[Вещество, заполняющее какое-л. пространство и обладающее определенными свойствами.]"


#### 2. Печь

ELMO для слова "печь" всё отнёc к одному классу, AdaGram справилась с этим немного лучше, но тоже часто ошибается.

In [91]:
data[(data['word']=='печь') & (~data['sense_elmo_text'].isna()) & (~data['sense_ada_text'].isna())].sample(5)

,word,text,sense_ada_id,sense_ada_text,sense_elmo_id,sense_elmo_text
403,печь,"Она включала лампочку за кухонной перегородкой и тихо , вежливо , стараясь не шуметь , топила русскую печь , ходила доить козу ( все животы её были -- одна эта грязно-белая криворогая коза ), по воду ходила и варила в трех чугунках : один чугунок -- мне , один -- себе , один -- козе .",4,"[Обдавать сильным жаром, зноем]",1,"[Приготовлять пищу сухим нагреванием на жару, Каменное или металлическое сооружение для отопления помещений, приготовления горячей пищи., Обдавать сильным жаром, зноем]"
439,печь,"Мы потушили наши "" печи "", а излишки "" дров "" некуда девать .",2,[Приготовлять пищу сухим нагреванием на жару],1,"[Приготовлять пищу сухим нагреванием на жару, Каменное или металлическое сооружение для отопления помещений, приготовления горячей пищи., Обдавать сильным жаром, зноем]"
400,печь,"Разговаривая со мной , она так и лежала на печи ничком , без подушки , головой к двери , а я стоял внизу .",4,"[Обдавать сильным жаром, зноем]",1,"[Приготовлять пищу сухим нагреванием на жару, Каменное или металлическое сооружение для отопления помещений, приготовления горячей пищи., Обдавать сильным жаром, зноем]"
417,печь,"Обычно прабабка проводила время , прислонясь в кухне спиною к печи и глядя , как кто-нибудь ест .",4,"[Обдавать сильным жаром, зноем]",1,"[Приготовлять пищу сухим нагреванием на жару, Каменное или металлическое сооружение для отопления помещений, приготовления горячей пищи., Обдавать сильным жаром, зноем]"
427,печь,"Натопили печь , напекли картошки , женщины приготовили гуся с яблоками , во дворе мы нарядили огромную красавицу ёлку и стали пускать фейерверки , а на следующее утро катались на санках с горы .",3,"[Каменное или металлическое сооружение для отопления помещений, приготовления горячей пищи.]",1,"[Приготовлять пищу сухим нагреванием на жару, Каменное или металлическое сооружение для отопления помещений, приготовления горячей пищи., Обдавать сильным жаром, зноем]"


#### 3. Ключ

Со словом ключ возникла другая проблема: появились значения, которых нет в словаре. Такие, как в третьем примере ниже ("ключ шифрования" и тд) или выражения типа "в таком ключе он произнёс эти слова". И, как уже говорилось, ELMO слишком много значений объединило в одно. В целом, примеров было в основном три типа: шифрование, ключ от замка и "в положительном ключе".

In [35]:
data[(data['word']=='ключ') & (~data['sense_elmo_text'].isna()) & (~data['sense_ada_text'].isna())].sample(5)

,word,text,sense_ada_id,sense_ada_text,sense_elmo_id,sense_elmo_text
144,ключ,"Злоумышленники проникли в жилище в Староконюшенном переулке, подобрав ключи.",1,[Металлическое приспособление для запирания и отпирания замка],4,[Металлическое приспособление для запирания и отпирания замка]
19,ключ,"Закрывшись , он втиснул клин между дверью и рычагом замка и несколько раз ударил по нему ладонью -- теперь проводник не смог бы отпереть дверь с той стороны даже своим ключом .",1,[Металлическое приспособление для запирания и отпирания замка],4,[Металлическое приспособление для запирания и отпирания замка]
30,ключ,"Кроме того , AES поддерживает более длинные , 192- и 256-разрядные ключи .",2,"[Орудие, приспособление для укрепления или отвинчивания чего-л., для приведения в действие механизмов различного рода, Знак в начале нотной строки, условно указывающий на ноту, по высоте которой устанавливается высотное положение других нот (муз.), Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи]",4,[Металлическое приспособление для запирания и отпирания замка]
158,ключ,26 июня губернатор региона Андрей Воробьев вручил ключи от новых квартир детям-сиротам в Химках.,4,"[Бьющий из земли источник, родник, Верхний или средний камень, которым замыкается свод или арка здания]",5,"[Орудие, приспособление для укрепления или отвинчивания чего-л., для приведения в действие механизмов различного рода, Средство, возможность для разгадки, понимания чего-л., для овладения чем-л., Пункт, место, важные в военном отношении, овладение которыми открывает доступ куда-л., обеспечивает победу, Знак в начале нотной строки, условно указывающий на ноту, по высоте которой устанавливается высотное положение других нот (муз.), Бьющий из земли источник, родник, Выключатель для быстрого замыкания и разрыва цепи передатчика при телеграфной и радиотелеграфной связи, Верхний или средний камень, которым замыкается свод или арка здания]"
104,ключ,"— сказал актер, добавив, что кольца висят не только на связках ключей.",1,[Металлическое приспособление для запирания и отпирания замка],4,[Металлическое приспособление для запирания и отпирания замка]


#### 4. Работа

В этом случае и AdaGram и ELMO много значений отнесли к какой-то одной группе. Поэтому попаданий достаточно много.

In [39]:
data[(data['word']=='работа') & (~data['sense_elmo_text'].isna()) & (~data['sense_ada_text'].isna())].sample(5)

,word,text,sense_ada_id,sense_ada_text,sense_elmo_id,sense_elmo_text
2880,работа,"Американский телеканал заявил о строительных работах на 40 бункерах в районе города Приморска, где находится второй по величине порт России на Балтийском море, и возведении дорожной инфраструктуры на подъезде к военной базе «Чкаловск» к северу от Калининграда, где базируются самолеты морской авиации Балтийского флота.",1,"[деятельность, занятие, труд, Та или иная деятельность по созданию, изготовлению, обработке чего-л, Служба, занятие, труд на каком-л. предприятии, в каком-л. учреждении, То, что подлежит обработке, находится в процессе изготовления, Продукт труда, изделие, произведение, Качество или какой-л. способ, манера исполнения, изготовления]",2,"[Та или иная деятельность по созданию, изготовлению, обработке чего-л, Качество или какой-л. способ, манера исполнения, изготовления, Мера действия силы на тело, перемещаемое этой силой по определенному пути]"
2821,работа,Экстренные службы ведут поисковые работы.,1,"[деятельность, занятие, труд, Та или иная деятельность по созданию, изготовлению, обработке чего-л, Служба, занятие, труд на каком-л. предприятии, в каком-л. учреждении, То, что подлежит обработке, находится в процессе изготовления, Продукт труда, изделие, произведение, Качество или какой-л. способ, манера исполнения, изготовления]",5,"[деятельность, занятие, труд, Служба, занятие, труд на каком-л. предприятии, в каком-л. учреждении, То, что подлежит обработке, находится в процессе изготовления, Продукт труда, изделие, произведение]"
1855,работа,"Поскольку жена у меня спортивная, она сама выполняла часть работы.",4,"[Мера действия силы на тело, перемещаемое этой силой по определенному пути]",5,"[деятельность, занятие, труд, Служба, занятие, труд на каком-л. предприятии, в каком-л. учреждении, То, что подлежит обработке, находится в процессе изготовления, Продукт труда, изделие, произведение]"
2351,работа,Россия готова участвовать в работе по восстановлению экономики и социальной сферы Ливии.,1,"[деятельность, занятие, труд, Та или иная деятельность по созданию, изготовлению, обработке чего-л, Служба, занятие, труд на каком-л. предприятии, в каком-л. учреждении, То, что подлежит обработке, находится в процессе изготовления, Продукт труда, изделие, произведение, Качество или какой-л. способ, манера исполнения, изготовления]",2,"[Та или иная деятельность по созданию, изготовлению, обработке чего-л, Качество или какой-л. способ, манера исполнения, изготовления, Мера действия силы на тело, перемещаемое этой силой по определенному пути]"
4434,работа,"На Тагульском месторождении продолжается эксплуатационное бурение (за 1 полугодие пробурено 19 скважин), также ведутся работы по строительству объектов обустройства месторождения (установка подготовки нефти, кустовые площадки, нефтепроводы и прочие).",1,"[деятельность, занятие, труд, Та или иная деятельность по созданию, изготовлению, обработке чего-л, Служба, занятие, труд на каком-л. предприятии, в каком-л. учреждении, То, что подлежит обработке, находится в процессе изготовления, Продукт труда, изделие, произведение, Качество или какой-л. способ, манера исполнения, изготовления]",2,"[Та или иная деятельность по созданию, изготовлению, обработке чего-л, Качество или какой-л. способ, манера исполнения, изготовления, Мера действия силы на тело, перемещаемое этой силой по определенному пути]"


#### 5. Мешать

Возникла проблема с тем, что не оказалось контекстов кроме "являться помехой". Так как использовалась кластеризация, модели пришлось разделить все примеры на разные классы, из-за этого примеры разошлись по разным группам, но на самом деле должны относиться к одной.
Но при этом обе модели смогли хорошо разнести значения по разным группам.

In [49]:
data[(data['word']=='мешать') & (~data['sense_elmo_text'].isna()) & (~data['sense_ada_text'].isna())].sample(5)

,word,text,sense_ada_id,sense_ada_text,sense_elmo_id,sense_elmo_text
381,мешать,"Однако дальнейшая деятельность стала мешать украинским властям, из-за чего она оказалась за решеткой уже на родине, считает она.",1,"[Быть, являться помехой, препятствием в чем-л., Соединять в одно что-л. разнородное]",1,"[Соединять в одно что-л. разнородное, Путать, принимать одно за другое.]"
285,мешать,"Среди факторов, мешающих отдыхать, туристы также отметили брань, курение, навязчивых соседей, невкусную еду и надоедливых продавцов.",1,"[Быть, являться помехой, препятствием в чем-л., Соединять в одно что-л. разнородное]",1,"[Соединять в одно что-л. разнородное, Путать, принимать одно за другое.]"
250,мешать,Первый / когда коммерческие структуры говорят о том / что им никакого государства вообще не нужно / и чем меньше оно им мешает / тем лучше им жить .,1,"[Быть, являться помехой, препятствием в чем-л., Соединять в одно что-л. разнородное]",2,"[Перемешивать, размешивать при помощи ложки, мешалки и т. п.]"
337,мешать,"При этом мешать выполнить боевую задачу, если она не выгодна партнёрам, тебе будут всеми способами.",1,"[Быть, являться помехой, препятствием в чем-л., Соединять в одно что-л. разнородное]",3,"[Быть, являться помехой, препятствием в чем-л.]"
173,мешать,Что ж вам мешает - то ?,3,"[Путать, принимать одно за другое.]",2,"[Перемешивать, размешивать при помощи ложки, мешалки и т. п.]"
